In [1]:
import numpy as np
import pandas as pd

In [6]:
mapping = pd.read_csv("numbermap.csv", header=None, index_col=0, squeeze=True).to_dict()

In [7]:
mapping

{'0': 'zero',
 '1': 'one',
 '2': 'two',
 '3': 'three',
 '4': 'four',
 '5': 'five',
 '6': 'six',
 '7': 'seven',
 '8': 'eight',
 '9': 'nine',
 '10': 'ten',
 '11': 'eleven',
 '12': 'twelve',
 '13': 'thirteen',
 '14': 'fourteen',
 '15': 'fifteen',
 '16': 'sixteen',
 '17': 'seventeen',
 '18': 'eighteen',
 '19': 'nineteen',
 '20': 'twenty',
 '21': 'twenty one',
 '22': 'twenty two',
 '23': 'twenty three',
 '24': 'twenty four',
 '25': 'twenty five',
 '26': 'twenty six',
 '27': 'twenty seven',
 '28': 'twenty eight',
 '29': 'twenty nine',
 '30': 'thirty',
 '31': 'thirty one',
 '32': 'thirty two ',
 '33': 'thirty three',
 '34': 'thirty four',
 '35': 'thirty five',
 '36': 'thirty six',
 '37': 'thirty seven',
 '38': 'thirty eight',
 '39': 'thirty nine',
 '40': 'forty ',
 '41': 'forty one',
 '42': 'forty two',
 '43': 'forty three',
 '44': 'forty four',
 '45': 'forty five',
 '46': 'forty six',
 '47': 'forty seven',
 '48': 'forty eight',
 '49': 'forty nine',
 '50': 'fifty ',
 '51': 'fifty one',
 '52':

In [216]:
mapping[''] = ''
mapping['%'] = 'percent'

In [130]:
tens = {
# '100': 'hundred',
 '': '',
 '1000': 'thousand',
#  '10000': 'ten thousand',
#  '100000': 'hundred thousand',
 '1000000': 'million',
#  '10000000': 'ten million',
#  '100000000': 'hundred million',
 '1000000000': 'billion'}

tens_keys = list(tens.keys())

In [131]:
tens_keys

['', '1000', '1000000', '1000000000']

In [228]:
def read_whole_number(string):
    rev = string[::-1]
    chunks = []
    for i in range(0,len(rev),3):
        chunks.append(rev[i:i+3][::-1])
    chunks = chunks[::-1]
    result = ""
    tens_idx = len(chunks)-1
    
    for chunk in chunks:
        if len(chunk) < 3:
            result += mapping[chunk] + ' '
            result += tens[tens_keys[tens_idx]] + ' '
        elif chunk == '000':
            result = result
        else:
            result += process_chunk(chunk) + ' '
            result += tens[tens_keys[tens_idx]] + ' '
        tens_idx -= 1
    return result
    
def process_chunk(string):
    hundreds = string[0]
    if hundreds == '0':
        hundred_string = ''
        tens = str(int(string[1:]))
        if tens == '0':
            return ''
        else:
            return mapping[tens]
    else:
        hundred_string = hundreds
        tens = str(int(string[1:]))
        if tens == '0':
            return mapping[hundred_string] + ' ' + mapping['100']
        else:
            return mapping[hundred_string] + ' ' + mapping['100'] + ' ' + mapping[tens]
    

In [229]:
def read_all(string):
    res = ""
    for n in string:
        res += mapping[n] + ' '
    return res

In [312]:
def read_number(string):
    if string.isnumeric() and 1500 < int(string) < 2100:
        if string[1:-1] == '00':
            return read_whole_number(string)
        elif string[-2:] == '00':
            return read_whole_number(string[:2]) + 'hundred'
        else:
            if string[2:][0] == '0':
                return read_whole_number(string[:2]) + 'zero ' + read_whole_number(string[3:])
            else:
                return read_whole_number(string[:2]) + read_whole_number(string[2:])
    elif mapping['currency_symbol'] in string:
        return read_number(string[1:]) + mapping['currency'] + 's'
    elif '-' in string:
        return mapping['-'] + read_number(string[1:])
    elif '.' in string:
        if string[-1] == '.':
            return read_number(string[:-1])
        s1, s2 = string.split('.')
        return read_whole_number(s1) + mapping['.'] + ' ' + read_all(s2)
    elif '+' in string: #phone number?
        return mapping['+'] + ' ' + read_all(string[1:])
    elif '%' in string:
        return read_number(string[:-1]) + ' ' + mapping['%']
    else:
        return read_whole_number(string)

In [313]:
def process_final(string):
    return ' '.join([x for x in read_number(string).split(' ') if x!= ''])

In [314]:
for _ in range(10):
    n = np.random.randint(1500, 2100)
    print(n, process_final(str(n)))

1534 fifteen thirty four
1882 eighteen eighty two
1700 seventeen hundred
1611 sixteen eleven
1803 eighteen zero three
1818 eighteen eighteen
1623 sixteen twenty three
1620 sixteen twenty
1538 fifteen thirty eight
1904 nineteen zero four


In [315]:
import string
def process_text(s):
    tokens = s.split(' ')
    tokens = [token for token in tokens if token not in string.punctuation]
    sentence = []
    for token in tokens:
        if any(i.isdigit() for i in token):
            sentence.append(process_final(token))
        else:
            sentence.append(token)
    return ' '.join(sentence)

In [316]:
process_text('Athiya Deviyani is 22 years old and is born in 1999.')

'Athiya Deviyani is twenty two years old and is born in nineteen ninety nine'